# Tratamento dos arquivos de treinamento e teste

Nesse notebook trataremos os arquivos `trasact_train.csv`, `transact_class.csv` e `realclass_t1.csv`.

O objetivo é preparar os dados na forma de CSV para utiliza-los na fase seguinte, escolha de paramêtros dos algoritmo de classificação.

Ao final do processo serão gerados 3 pares de arquivos, cada par referente a uma estratégia para tratamento de missing values.

### Importar as bibliotecas necessárias

In [2]:
import pandas as pd
import numpy as np

### Ler os arquivos csv

In [42]:
transact_train = pd.read_csv('data/transact_train.csv', sep='|', na_values='?')
transact_class = pd.read_csv('data/transact_class.csv', sep='|', na_values='?')
realclass_t1 = pd.read_csv('data/realclass_t1.csv', sep='|', na_values='?')

# Remover os dados de ID do usuário
transact_train.drop('customerNo', axis=1, inplace=True)
transact_class.drop('customerNo', axis=1, inplace=True)

### Diminuir a granularidade dos dados

In [44]:
def reduceGranularity(data):
    ant = data['sessionNo'][0]
    indexes = []
    reduced_data = pd.DataFrame()
    for index, row in data.iterrows():
        if row['sessionNo'] != ant:
            indexes.append(index)      
        ant = row['sessionNo']
    for index in range(0, len(indexes)):
        indexes[index] -= 1
    indexes.append(len(data) - 1)
    reduced_data = data.iloc[indexes].set_index('sessionNo')
    return reduced_data


In [48]:
training = reduceGranularity(transact_train)
testing = reduceGranularity(transact_class)

### Substituir missing values

In [69]:
# Substitui valores pela moda
def fillNaByMode(data):
    new_data = data.copy()
    for col in new_data:
        new_data[col].fillna(value=new_data[col].mode().iloc[0], inplace=True)
    return new_data
 
# Substitui valores inteiros pela média e valores nominais pela moda
def fillNaByMean(data):
    new_data = data.copy()
    for col in new_data:
        if data[col].dtype == 'object':
            new_data[col].fillna(value=new_data[col].mode().iloc[0], inplace=True)
        else:
            new_data[col].fillna(value=new_data[col].mean(), inplace=True)
    return new_data

In [73]:
training_mean = fillNaByMean(training)
training_mode = fillNaByMode(training)
training_no_na = training.dropna()

### Salvar os dados em arquivos CSV

In [76]:
training_mean.to_csv('data/training_mean.csv')